# Day 18

## Part 1

Given a map like this:
```
#################
#i.G..c...e..H.p#
########.########
#j.A..b...f..D.o#
########@########
#k.E..a...g..B.n#
########.########
#l.F..d...h..C.m#
#################
```

Where `@` is us, `x` is a key opening a door `X`, `#` a wall, we need to find the shortest path to get all the keys.

In [3]:
def read_map(txt):
    return [[v for v in line] for line in txt.splitlines() if line]

In [4]:
def count_keys(map_):
    keys = 0
    for line in map_:
        for v in line:
            if v.islower():
                keys += 1
            
    return keys

In [5]:
def find_origin(map_):
    for y, line in enumerate(map_):
        for x, v in enumerate(line):
            if v == "@":
                return (x, y)

In [6]:
def compute_path(map_):
    """Compute a path.
    
    A path is a list of tuples `(x, y, keys)`, `keys` being an ordered tuple
    of keys.
    At each intersections, it tries every path.
    
    Returns a list of tuples `(path, keys)`.
    """
    x, y = find_origin(map_)
    path = [(x, y, [])]
    candidates = [path]

    result = []
    keys_count = count_keys(map_)
    min_found = float('+inf')

    while True:
        candidates.sort(key=lambda p: (len(p), -len(p[-1][2])))
        try:
            path = candidates.pop(0)
            #print(path)
        except IndexError:
            return result

        if len(path) >= min_found:
            continue
        
        px, py, keys = path[-1]
        for x, y in (px - 1, py), (px + 1, py), (px, py - 1), (px, py + 1):
            if (x, y, keys) in path:
                # We were already here we the same set of keys, this is a dead end,
                # let's stop.
                continue
            elif map_[y][x] == "#":
                continue
            elif map_[y][x].isupper():
                # It's door!
                key = map_[y][x].lower()
                if key in keys:
                    # We have the key, let's use it and go forward.
                    subpath = path + [(x, y, keys)]
                    candidates.append(subpath)
                else:
                    # We don't have this door's key, we can't go through it.
                    continue
            elif map_[y][x].islower() and map_[y][x] not in keys:
                # It's a key! Pick it up!
                subkeys = sorted(keys + [map_[y][x]])
                subpath = path + [(x, y, subkeys)]

                if len(subkeys) == keys_count:
                    # We found all the keys!
                    if len(subpath) < min_found:
                        min_found = len(subpath)
                    result.append(subpath)
                    continue

                candidates.append(subpath)
            else:
                # Empty path, let's go
                subpath = path + [(x, y, keys)]
                candidates.append(subpath)

In [21]:
from collections import defaultdict, namedtuple
from dataclasses import dataclass
from typing import List, Tuple

Map = namedtuple("Map", ("root_node",))
Node = namedtuple("Node", ("coordinates", "links",))
Link = namedtuple("Link", ("length", "node",))

@dataclass
class Node:
    coordinates: Tuple[int, int]
    links: List[Link]
        
@dataclass
class Link:
    node: Node
    length: int = 0

@dataclass
class Map:
    root_node: Node

def preprocess_map(flat_map, map_=None, node=None, previous_node=None):
    """Prepocess a flat map.
    
    Corridors are removed. Returns à tree of inserections.
    
    `map_` is the constructing map. `node` is the current node. `previous` is the
    node we are coming from.
    """
    if node is None:
        node = Node(
            coordinates=find_origin(flat_map),
            links=[],
        )
        
    if map_ is None:
        map_ = Map(root_node=node)
    
    if previous_node is None:
        previous_node = Node(
            coordinates=node.coordinates,
            links=[]
        )

    candidates = [(node, previous_node)]

    while True:
        if len(candidates) == 0:
            return map_
        
        node, previous_node = candidates.pop()
        link = Link(node=node, length=1)
        previous_node.links.append(link)
        previous = previous_node.coordinates
        px, py = node.coordinates
        #print(f"starting at {px}, {py}")

        while True:
            new_directions = []
            for x, y in (px - 1, py), (px + 1, py), (px, py - 1), (px, py + 1):
                if (x, y) == previous:
                    continue

                try:
                    if flat_map[y][x] != "#":
                        new_directions.append((x, y))
                        flat_map[y][x] = "#"
                except IndexError:
                    pass

            if len(new_directions) == 0:
                # It's a cul-de-sac
                #print("it's a cul-de-sac")
                break
            elif len(new_directions) == 1:
                # Let's continue
                #print(f"let's continue to {new_directions[0]}")
                link.length += 1
                previous = (px, py)
                node.coordinates = new_directions[0]
                px, py = new_directions[0]
                continue
            else:
                # The current node is an intersection
                #print("intersection!")
                for p in new_directions:
                    new_node=Node(
                        coordinates=p,
                        links=[],
                    )
                    candidates.append((new_node, node))
                    #preprocess_map(flat_map, map_=map_, node=new_node, previous_node=node)

                break

In [22]:
preprocess_map(read_map("""
#########
#b.A.@.a#
#########
"""))

Map(root_node=Node(coordinates=(5, 1), links=[Link(node=Node(coordinates=(7, 1), links=[]), length=2), Link(node=Node(coordinates=(1, 1), links=[]), length=4)]))

In [23]:
preprocess_map(read_map("""
########################
#...............bCCDD.f#
#.######################
#.....@.aBB.c.dAA.eFF.g#
########################
"""))

Map(root_node=Node(coordinates=(6, 3), links=[Link(node=Node(coordinates=(22, 3), links=[]), length=16), Link(node=Node(coordinates=(22, 1), links=[]), length=28)]))

In [30]:
preprocess_map(read_map("""
#################
#i.GGGc...eHHH.p#
########.########
#j.AAAb...fDDD.o#
########@########
#k.EEEa...gBBB.n#
########.########
#l.FFFd...hCCC.m#
#################
"""))

Map(root_node=Node(coordinates=(8, 4), links=[Link(node=Node(coordinates=(8, 5), links=[Link(node=Node(coordinates=(8, 7), links=[Link(node=Node(coordinates=(15, 7), links=[]), length=7), Link(node=Node(coordinates=(1, 7), links=[]), length=7)]), length=2), Link(node=Node(coordinates=(15, 5), links=[]), length=7), Link(node=Node(coordinates=(1, 5), links=[]), length=7)]), length=1), Link(node=Node(coordinates=(8, 3), links=[Link(node=Node(coordinates=(8, 1), links=[Link(node=Node(coordinates=(15, 1), links=[]), length=7), Link(node=Node(coordinates=(1, 1), links=[]), length=7)]), length=2), Link(node=Node(coordinates=(15, 3), links=[]), length=7), Link(node=Node(coordinates=(1, 3), links=[]), length=7)]), length=1)]))

In [31]:
preprocess_map(read_map("""
#################################################################################
#.#...#.....#...............#.....#v....#.........#p....#...#...................#
#.#.#.#.#.#.#M#############.#.###.#####.#####.###.#.###.#.#.#################.#.#
#..n#...#.#...#.#...........#...#.......#.....#.....#.#...#...#.......#...#...#.#
#########.#####.#.###########.#.#######.#.###########.#######.#C#####.#.#.#.###.#
#.........#.#.....#.........#.#.#.......#...........#...........#...#.#.#...#...#
#.#########.#.#########.#####.#W###.###############.###.#########.###.#.#####.###
#.Y...#...#...#.......#z......#...#.#...#...#.....#..d#.#.#..x..#...#.#.#...#.#.#
#####.#.#.#.###.#.###.#####.#####.#.#.#.#.#.#.###.###.#.#.#.###.#.#.#.#.#.#.#.#.#
#.....#.#...#...#.#.#.....#...#...#.#.#.#.#...#.......#...#.#...#.#.....#.#...#.#
#.#######.#######.#.#####.#####.#####.#.#.#############.###.#.###.#########.###.#
#...#...#.#...#...#.#...#.....#...B...#.#.....#...#.....#...#.#.....#....t#.....#
#.#.#.#.#.#.#.#.###.#.#.#####.#.#######.#.###.#.#.###.###.###.#.#####.###.#####N#
#.#...#.#...#.#.#.....#...#...#.#...#...#...#.#.#...#.#...#...#...#...#.#...#.#.#
#.#####.#####.#.#########.#.###.#.#.#.#.###.#.#.###.###.###.#######.###.###.#.#.#
#.#...#.#.......#...#.....#...#...#.#.#.#.#.#...#.#.....#...#.....#.#...#...#.#.#
#.#.###.#########.#.#.###.###.#####.#.#.#.#.#####.#######.###.###.#.#.###.###.#.#
#.#...#.#...#.....#.#...#...#.#...#.#.#.#.#.#.....#.....#...#.#.#...#...#.#...#.#
#.###.#.#.#.#.#####.#.#.#####.#.###.#.###.#.#.###.###.#.###.#.#.#####.#.#J#E###.#
#.....#.#.#...#...#.#.#.....#.#.....#...#.#.#.#.......#.#...#.#...#...#...#.....#
#######.#.#####.###.#######.#.#########.#.#.#.#########.#.###.#.#.#.#########.###
#.......#.#.....#.I.#.....#.............#...#...#.....#.#...#.#.#.#.........#...#
#.#######.###.###.#.#####.#.###########.#.###.#.###.###.###.#####.#########.###.#
#.......#...#.#...#.....#.#.#.......#...#.#...#.#...#.....#.........#.....#.#...#
#.#####.###.#.#.#######.#.###.#####.#.###.#####.#.###.#######.#####.#.###.#.###.#
#.#...#...#.#.#.......#.#.....#.#...#...#.#...#.#...#u#.....#...#...#.#.#.#...#.#
#.###.#.###.#.#######.#.#######.#.#####.#.#.#.#.#.#.#.#.#.#.###.#####.#.#.###.#.#
#...#.#...#.....#.....#.#.....#...#...#.#...#.#.#.#...#.#.#...#.#.....#.#.#...#.#
###.#.###.#####.#.#####.#.#.#.#.###.#.#.#####.#.#.#####.#.#.###.#.#####.#.#.###.#
#...#...#.......#.#.....#.#.#.#.....#.#.#.....#.#.#.....#.#.#...#.....#.#.#.#..k#
#.###.###########.#.#######.#.#######.#.#.#####.###.#####.###.#.#####.#.#.#.#.###
#...#...........#.#.....#...#.......#.#.#...#.........#...#...#.#.....#.#...#.#.#
###.#.###.###.###.#####.#.#.#######.#.#.###.#.#########.#.#.###.#.#####.#####.#.#
#.#.#...#...#.#...#...#.#.#.#.....#.#.#.#.#.#...#.......#.#.#...#...#.....#...#.#
#.#.###.###.#.#.###.#.#.#.###.###.###.###.#.#####.#######.#.#.#####.#.#.#.#.###.#
#.#.#.....#.#.#.....#g#...#...#.#...#...#.#.......#...#...#.#...#.#.#.#.#.#.....#
#.#.#######.#.#######.###.#.###.###.###.#.#########.###.###.###.#.#.#.#.#######.#
#...#.....#.#.#.....#.#...#.......#.#...#.....#.......#.#.#...#...#.#.#.......#.#
#.###.###.#.#.#.###.#.###########.#.#.###.###.#.#####.#.#.###.#####.#.#######.#.#
#.....#.....#...#...#.A...........#.........#.......#.R.....#.......#.......#...#
#######################################.@.#######################################
#.....#.........#...........#......r#...........#.#...#.............#.......#...#
###.###.#.#######.#.#######.#.#####.###.#.#.###.#.#.#.#.#######.###.#.#####.###.#
#...#...#.........#...#...#.#.....#.....#.#...#...#.#...#...#...#.#.#.....#.....#
#.###.###############.#.###.#.###.#####.#.###.#####.#######.#.###.#.#####.#######
#.....#.......#.#.....#...#.#.#...#.....#.#.#.#.....#.......#.#...#.....#.......#
#.#######.###.#.#.#######.#.###.#.#######.#.#.#.#########.###.#.#######G#.#####.#
#.....#...#.#...#.#.......#...#.#.#.....#...#.#.#.....#..a#...#.#.....#.#i....#.#
#####.#.###.###.#.###.#.#####.#.###.###.#####.#.###.#.#.#.#.###.#.###.#.#######.#
#...#.#.#.....#.#...#.#.#.....#.....#...#...#.......#...#.#.#.....#.#.........#.#
#.#.#.#.#.#####.###.#.###.###########.###.#.#.#############.#.#####.#########.#.#
#.#.#.#.#.......#...#.....#.....#...#...#.#.#.#.............#.......#...#.....O.#
#.#.#.#.#########.#######.#.###.#.#.###.#.#.###.###########.#########.#.#######.#
#.#.#e#.........#.#.....#...#...#.#.....#.#.#...#...........#.......#.#.....#...#
###.#.#########.#.#.###.#####P###.#######.#.#.#########.#####.#####.#.#####.#.###
#...#.......#...#...#.#.#...#...#...#...#.#...#.......#.#w....#f..#...#...#.#...#
#.#.#######.#.#######.#.#.#####.#.#.#.#.#.#.###.#####.###.###.#.#.#####.#.#.#####
#.#.#...#...#.........#.#...#...#.#...#.#.#.#.......#.#...#...#.#...#...#.#.#...#
#.#.#.###.#######.###.#.#.#.#.###.#####.#.###.#######.#.#######.###.#.###.#.#.#.#
#.#...#...#.....#...#.#.#.#...#..o..#...#...#...#...#.#...#...F.#...#...#.#...#.#
#.#####.###.###.###.###.#####.#####.#.#.#.#.#.###.#.#.###.#.#####L###.#.#.#####.#
#.......#...#.#.#...#.#.....#.....#.#.#.#.#...#...#.#.#...#...#...#...#.#.....#.#
#K#######.#.#.#.#.###.#####.#####X###.#.#######.###.#.#.#######.#####.#.#######.#
#....j..#.#.#.....#...#.....#...#...#.#.#.......#...#.#.....#...#.S.#.#.........#
#######.###.#####.#.###.#####.#.###.#.###.#######.###.#####.#.###.#.#.###########
#.....#...#.....#.#...#.......#...#.#...#.....#.#.#.......#.#l....#.#.#...#.....#
#T#.#####.#####.#####.#.#########.#.###.#.###.#.#.#####.#.#.#######.#.#.###.###.#
#.#.....#.#.....#.....#.#.......#.#.#...#c#.#.#.....#...#.#...Z.#...#.#.....#...#
#.###.###.#.#.###.#####.#.#####.#.#.#.#.#.#.#.#####.#.#######.###.###.#.#####.#.#
#...#...#...#.#...#...#...#...#.#.#.#.#.#...#.....#...#.....#.#s..#...#.#.#...#.#
###.###.#.#####.###.###.###.#.###.#.#.###.#######.###.#.#.###.#.###.###Q#.#.#####
#.#.#.#.#.#...#...#...#.#...#.....#.#...#.#.....#.#...#.#.#...#.V.#.#...#.#.....#
#.#.#.#.#.#.#.###.#.#.#.#.#########.###.#.#.###.#.#####.#.#.#####.#.#.###.#####.#
#...#.#...#.#...#.#.#.#.#...#...#...#...#.#...#y..#...#.#.#.......#.#.#.......#.#
#.###.#####.###.#.###.#.###.#.###.###.#.#####.#####.#.#.#.#########.#.#.#.#####.#
#.#.....#...#.#...#...#.#...#.#.D.#.U.#.#.....#...#.#.#.#.......#...#..b#.#.....#
#.#####.#.###.#####.###.#.###.#.###.###.#.#####.#.#.#.#.#######.#.#######.#.###.#
#.H...#.#...#...........#.#...#.#.....#.#m......#...#...#.......#.#...#...#.#...#
#####.#.###.#############.#.#.#.#####.#.#.###############.#######.#.#.#####.#.###
#..q......#...............#.#.........#.#...............#...........#.......#..h#
#################################################################################
"""
))

Map(root_node=Node(coordinates=(40, 40), links=[Link(node=Node(coordinates=(40, 41), links=[Link(node=Node(coordinates=(41, 41), links=[Link(node=Node(coordinates=(43, 45), links=[]), length=10), Link(node=Node(coordinates=(43, 41), links=[Link(node=Node(coordinates=(45, 49), links=[Link(node=Node(coordinates=(45, 51), links=[]), length=2), Link(node=Node(coordinates=(47, 49), links=[Link(node=Node(coordinates=(63, 41), links=[Link(node=Node(coordinates=(59, 51), links=[Link(node=Node(coordinates=(55, 53), links=[Link(node=Node(coordinates=(55, 55), links=[]), length=2), Link(node=Node(coordinates=(49, 53), links=[]), length=6)]), length=6), Link(node=Node(coordinates=(43, 55), links=[Link(node=Node(coordinates=(43, 57), links=[]), length=2), Link(node=Node(coordinates=(41, 59), links=[Link(node=Node(coordinates=(41, 61), links=[]), length=2), Link(node=Node(coordinates=(45, 59), links=[Link(node=Node(coordinates=(47, 57), links=[Link(node=Node(coordinates=(53, 65), links=[Link(node=No

In [4]:
def get_min_len(map_):
    map_ = [[v for v in line] for line in map_.splitlines() if line]
    result = compute_path(map_)
    keys_count = count_keys(map_)

    min_path = None
    for path in result:
        if len(path[-1][2]) != keys_count:
            continue

        if min_path is None:
            min_path = len(path)
        
    return min_path

In [10]:
get_min_len("""
#########
#b.A.@.a#
#########
""")

9

In [25]:
%%time

get_min_len("""
########################
#...............bCCDD.f#
#.######################
#.....@.aBB.c.dAA.eFF.g#
########################
""")  # 132

CPU times: user 5.25 ms, sys: 0 ns, total: 5.25 ms
Wall time: 5.14 ms


133

We get 133 because we can the origin.

In [30]:
%%time

get_min_len("""
#################
#i.GGGc...eHHH.p#
########.########
#j.AAAb...fDDD.o#
########@########
#k.EEEa...gBBB.n#
########.########
#l.FFFd...hCCC.m#
#################
""")

KeyboardInterrupt: 